# The Materials API

### Presented by: John Dagdelen

In this lesson, we will learn how to interact with the Materials Project database and go through some practical exaples of combining our own code with MP data to uncover new materials insights. We will do this through the Materials API (MAPI), which is an open API for accessing Materials Project data based on [Representational state transfer (REST)](https://en.wikipedia.org/wiki/Representational_state_transfer) principles.

In this module, we cover:

* The Materials Project API (MAPI).
* Getting your Materials Project API key.
* The `MPRester.query` method for accessing the MP database.
* A hands-on example of using the API and pymatgen to screen the database for interesting materials.
* The [mapidoc](https://github.com/materialsproject/mapidoc) (Materials Project documentation) repository.


***
## Section 0: Getting an API key

The first step to getting started with the API is to get an API key. API keys are unique identifiers that are used to track and control how the API is being used. 

To get yours, go to the dashboard page on the Materials Project website (https://materialsproject.org/dashboard). Click the 'Generate API key' button and copy the string under the button; this is your API key.

Paste your key in the line below, which will assign it the environment variable name MAPI_KEY.

In [ ]:
!pmg config --add PMG_MAPI_KEY [your API key]

***
## Section 1: The MPRester

In this section we will:

* Open the pymatgen.MPRester web documentation.
* Create our first instance of an MPRester object.
* Get our feet wet with calling a few of the MPRester's "specialty" methods.
* Introduce the powerful `query` method. 



#### Background and Documentation

REST is a widely used type of standardization that allows different computer systems to work together. In RESTful systems, information is organized into resources, each of which is uniquely identified via a uniform resource identifier (URI). Since MAPI is a RESTful system, users can interact with the MP database regardless of their computer system or programming language (as long as it supports basic http requests.)

[NOTE: Should we do an example using the requests library before or after introducing the MPRester?] NO.

To facilitate researchers in using our API, we implemented a convenient wrapper for it in the Python Materials Genomics (pymatgen) library called the `MPRester`. You can find the relevant pymatgen documentation for it [here](http://pymatgen.org/pymatgen.ext.matproj.html?highlight=mprester#pymatgen.ext.matproj.MPRester).



#### Starting up an instance of the MPRester

We'll create an instance of the MPRester object using our API key as an input argument.


However, we recommend that you use the “with” context manager to ensure that sessions are properly closed after usage:

***
## Section 2: Usign the MPRester and Pymatgen to Find Materials With Exotic Mechanical Properties

The SiO$_2$ polymorph $\alpha$-cristobalite [(mp-6945)](https://materialsproject.org/materials/mp-6945/) is one of the very few crystalline materials known to have a negative average Poisson's ratio, which means that its cross-section expands under tensile strain rather than contracting. This property can be extremly useful in a variety of applications such as scratch-resistant coatings and high-toughness ceramics. 

Why does $\alpha$-cristobalite exhibit this property while other materials do not? The prevailing hypothesis is that $\alpha$-cristobalite's negative Poisson's ratio is a result of its crystal structure. If that's the case, then perhaps we can find other materials with this exotic property by looking for materials with similar structures and then calculating their Poisson's ratios.

First, it might be nice to inspect $\alpha$-cristobalite's structure. The MPRester has a handy special method called `get_structure_by_material_id` that allows us to request the pymatgen structure object for a material by passing in its mpid. Let's use this method to get $\alpha$-cristobalite's structure and print it out. 

In [ ]:
ac_structure = [your code here]
print(ac_structure)

The MPRester also has a very powerful method called `query`, which allows us to perform sophisticated searches on the database. The `query` method uses MongoDB's [query syntax](https://docs.mongodb.com/manual/tutorial/query-documents/). In this syntax, query submissions have two parts: a set of criteria that you want to base the search on (in the form of a python dict), and a set of properties that you want the database to return (in the form of either a list or dict). 

The general structure of a MPRester query is:
                            
                            mpr.query(criteria={}, properties=[])


Example: Say that we want to get a list of the mpid and crystal system (cubic, tetragonal, etc) for every SiO$_2$ polymorph in the MP database. How would we construct the query?

The first step is to consult the Materials API Documentation [(mapidoc)](https://github.com/materialsproject/mapidoc) to find the right keywords. After that, we can fill in the query method's arguments `criteria` and `properties`.


(We find that the keys are "pretty_formula", "material_id", and "spacegroup.crystal_system" since crystal_system is a sub-property of spacegroup.)

You don't actually have to specify the argument names as long as the criteria are first and the properties second. 


Also, if you're querying on a simple property, such as the chemical formula, you can skip passing it as a dictionary. For example:

If we investigate the object that the query method returns, we find that it is a list of dicts. Furthermore, we find that the keys of the dictionaries are the very same keywords that we passed to the query method as the `properties` argument.

***
### Quick Aside About MongoDB Query Operators

Above, we specified the chemical formula SiO$_2$ for our query. This is an example of, the "specify" operator. However, MongoDB's syntax also includes other [query operators](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors), allowing us to bulid complex conditionals into our queries. 

A recent paper by McEnany et. al. proposes a novel ammonia synthesis process based on the electrochemical cycling of lithium ([link](http://pubs.rsc.org/en/content/articlelanding/2017/ee/c7ee01126a#!divAbstract)). As an excercise, let's use some of MongoDB's operators and ask the database for nitrides of alkali metals.

For convenience, the MPRester also accepts a simplified syntax for queries by chemical system. For example, the query we made above can be simplified to:

We can also perform the same query, but ask the database to only return compounds with energies above the hull less than 10 meV/atom by using the "less than" operator, "`$lt`". (The energy above the convex hull gives us a sense of how stable a compound is relative to other compounds with the same composition.) 

Now, let's get back to our example of finding materials with similar crystal structures to $\alpha$-cristobalite. 

***

For our search, we want to start with a set of structures that are:
* Computationally tractable (not too many sites)
* Not too unlikely to be synthesizable (small energy above hull, i.e. <100 meV)

Let's construct this query:

The next step is to compare all of these materials to $\alpha$-cristobalite. We'll need something that can tell us whether two structures are similar. Luckily for us, the pymatgen StructureMatcher does just that!

We know that the high-temperature phase of crisobalite, $\beta$-cristobalite [(mp-546794)](https://materialsproject.org/materials/mp-546794/), has a very similar structure to $\alpha$-cristobalite. Let's see if the structure matcher agrees.

Unfortunately, the default settings of the structure matcher are too strict for our purposes. We want a comparison engine that will return True if two structures are similar to each other, not just those that are exatly the same. 

To solve this problem, we can instantiate our comparison engine with looser tolerances and use a species-agnostic FrameworkComparator from pymatgen, which allows us to compare structures across different chemistries.

Just to make sure we haven't increased the tolerances too much, let's try it against a random compoud to make sure it's not matching un-similar structures. 

***

Imagine that we have an experimental colleague, Dr. Soren Tsarpinski, who is an expert at synthesizing vanadate compounds. We have a hunch that some of the vanadates coming out of Dr. Tsarpinski's lab might have similar structures to $\alpha$-cristobalite and therefore might have negative Poisson's ratios. Let's see if we're right: